In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

%pip install -q spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
%pip install -q pytorch-metric-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.9 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as transforms
import cv2
import torchvision.models as models
from pytorch_metric_learning import losses
import torch.optim as optim
from torch.optim import lr_scheduler
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class Config:
  spurious_correlation_strength  = 0.9
  spurious_feature_difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE
  label_noise = 0.001
  size = 28
  flip_probability = 0.5
  batch_size = 256
  backbone_pretrained = False
  projection_dim = 128
  T_max = 10
  lr = 5e-4
  min_lr = 1e-5
  weight_decay = 1e-6
  temperature = 0.5

## Dataset

In [6]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="train",
    label_noise=Config.label_noise
)
trainset.initialize()

valset = SpuCoMNIST(
    root="/data/mnist/",

    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="test",
    label_noise=Config.label_noise
)
valset.initialize()


100%|██████████| 9912422/9912422 [00:00<00:00, 118849887.93it/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 96753749.06it/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 39178637.20it/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 16171925.95it/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 10000/10000 [00:01<00:00, 7442.56it/s]


In [7]:

## Augmentation
from torchvision.transforms import v2

size = 28
train_transform = transforms.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    v2.RandomGrayscale(p=0.2),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    v2.ToImage(),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [8]:
class SpucoDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img = self.dataset[index][0]
        label = self.dataset[index][1]

        if self.transform is None:
          return img, label

        augmented_img = self.transform(img)
        return (img, augmented_img), label

In [9]:
train_dataset = SpucoDataset(trainset, train_transform)
val_dataset = SpucoDataset(valset, test_transform)

dataloaders = {'train': DataLoader(dataset=train_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=True),
         'eval': DataLoader(dataset=val_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=False)}

In [10]:
from IPython.display import display
for (k, v) in valset.group_partition.items():
  print(f"Group: {k}, Qty: {len(v)}")
  img = T.ToPILImage()(valset[v[0]][0])
  display(img)

Group: (3, 1), Qty: 397


Group: (1, 1), Qty: 409


Group: (0, 0), Qty: 423


Group: (2, 0), Qty: 375


Group: (0, 1), Qty: 423


...

Group: (1, 2), Qty: 408


Group: (3, 2), Qty: 397


Group: (1, 0), Qty: 409


Group: (4, 1), Qty: 397


Group: (4, 0), Qty: 397


In [11]:
T.ToPILImage()(valset[v[0]][0])

## Contrastive Loss

In [12]:
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#ntxentloss
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#selfsupervisedloss
criterion = losses.NTXentLoss(temperature=Config.temperature)
criterion = losses.SelfSupervisedLoss(criterion)

## Model

In [13]:
## https://github.com/berfukaraca/SSL-Cifar10-ContrastiveLearning/blob/main/colab_notebooks/SSL_SimCLR_Cifar10_CL.ipynb

class SimCLR_model(nn.Module):
    def __init__(self, base_encoder, projection_dim, n_features):
        super(SimCLR_model, self).__init__()
        self.n_features = n_features
        self.base_encoder = base_encoder

        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_dim, bias=False),
            nn.BatchNorm1d(projection_dim),
        )

    def forward(self, x_i):
        out = self.projectionHead(self.base_encoder(x_i))

        return out

In [14]:
## model and optimizer
model = models.resnet50()
# print(model)
clr_model = SimCLR_model(model, Config.projection_dim, 1000) ## output dimension of resnet18 is 1000
optimizer = optim.Adam(clr_model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
## cosine annealing lr: https://arxiv.org/pdf/1608.03983.pdf
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr)

clr_model.to(device)

SimCLR_model(
  (base_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


In [15]:
import os
def train(epoch):
    print('\nEpoch: %d' % epoch)
    clr_model.train()
    s=0
    tot_loss = 0
    total = 0
    batch_idx=0
    i=0
    batch_num =1
    val_loss = 0
    for batch_idx, (images, _) in enumerate(dataloaders['train']):
        batch_tot_loss = 0
        if (batch_num % 50 == 1):
          print("in batch", batch_num)
        batch_num=batch_num+1
        images, augmented_images = images[0], images[1]
        images, augmented_images = images.to(device), augmented_images.to(device)
        optimizer.zero_grad()

        images_emb = clr_model(images)
        augmented_images_emb = clr_model(augmented_images)
        loss = criterion(images_emb, augmented_images_emb)

        loss.backward()

        optimizer.step()
        tot_loss += loss.item()
        batch_tot_loss += loss.item()
        if (batch_idx % 200 == 1):
            print('batch loss=', batch_tot_loss)
    scheduler.step()



    # Validation
    model.eval()
    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(dataloaders['eval']):
            val_batch_loss = 0
            if (batch_num % 50 == 1):
                print("in batch", batch_num)
            batch_num=batch_num+1
            images, augmented_images = images[0], images[1]
            images, augmented_images = images.to(device), augmented_images.to(device)


            images_emb = clr_model(images)
            augmented_images_emb = clr_model(augmented_images)
            loss = criterion(images_emb, augmented_images_emb)



            val_loss += loss.item()
            val_batch_loss += loss.item()
        if (batch_idx % 200 == 1):
          print('batch loss=', val_batch_loss)

    print(f"epoch {epoch}train loss= {tot_loss/len(dataloaders['train'])}")
    print(f"epoch {epoch}val loss= {val_loss/len(dataloaders['train'])}")

    if epoch % 10 ==0:
        save_dir = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
        if not os.path.isdir(save_dir):
          os.mkdir(save_dir)
        modelstate_save_name = f'modelstate_bs_restart_r_{Config.spurious_correlation_strength}_LARGE_epoch{epoch}.pt'
        path_modelstate = os.path.join(save_dir, modelstate_save_name)
        torch.save(clr_model.state_dict(), path_modelstate)

        parameters_end = {
            'net': clr_model.state_dict(),
            'epoch': epoch,
            'loss': tot_loss
        }
        paramsend_save_name = f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_LARGE.pt'
        path_paramsend = os.path.join(save_dir, paramsend_save_name)
        torch.save(parameters_end, path_paramsend)


In [16]:
start_epoch=1
max_epoch=30

for epoch in range(start_epoch, start_epoch+max_epoch):
  train(epoch)


Epoch: 1


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 6.218566417694092
in batch 51
in batch 101
in batch 151
in batch 201
epoch 1train loss= 5.278068126516139
epoch 1val loss= 1.1078638967047347

Epoch: 2


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 5.108982086181641
in batch 51
in batch 101
in batch 151
in batch 201
epoch 2train loss= 5.083747072422758
epoch 2val loss= 1.1355329016421705

Epoch: 3


...

in batch 1
batch loss= 4.664175987243652
in batch 51
in batch 101
in batch 151
in batch 201
epoch 28train loss= 4.662353723607165
epoch 28val loss= 1.039944072987171

Epoch: 29


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 4.6434125900268555
in batch 51
in batch 101
in batch 151
in batch 201
epoch 29train loss= 4.649455409100715
epoch 29val loss= 1.0280698981690914

Epoch: 30


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 4.6659369468688965
in batch 51
in batch 101
in batch 151
in batch 201
epoch 30train loss= 4.650385301163856
epoch 30val loss= 1.0378785868908496


In [17]:
## Load the pre-trained model parameters
paramsend_save_name = f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_LARGE.pt'
save_dir = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
path_paramsend = path_paramsend = os.path.join(save_dir, paramsend_save_name)
checkpoint = torch.load(path_paramsend, map_location = torch.device(device))
loss= checkpoint['loss']
epoch = checkpoint['epoch']

start_epoch=epoch+1
clr_model.load_state_dict(checkpoint['net'])

<All keys matched successfully>

## PCA and 2D Scatter Plot

In [18]:
torch.cuda.empty_cache()

In [19]:
## https://www.kaggle.com/code/minc33/visualizing-high-dimensional-clusters

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA


torch.cuda.empty_cache()
device = 'cpu'
clr_model.to(device)

embeddings = []
targets = []

for batch_idx, ((images, _), labels) in enumerate(dataloaders['eval']):
  embeddings.append(clr_model(images).detach().cpu())
  targets.append(labels)
  torch.cuda.empty_cache()


# clr_model = None

embeddings = torch.cat(embeddings).detach().cpu().numpy()
targets = torch.cat(targets).cpu().numpy()

In [20]:
import pandas as pd
comp = 100
pca_2d = PCA(n_components=comp)
PCs_2d = pd.DataFrame(pca_2d.fit_transform(embeddings))
PCs_2d.columns = [f"PC{i+ 1}" for i in range(comp)]
pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')

In [21]:
pce = pca_2d.fit_transform(embeddings)

In [22]:
pca_2d.singular_values_

array([402.47046  , 394.7197   , 358.0145   , 316.558    , 313.92584  ,
       308.18018  , 297.66025  , 279.59506  , 244.0405   , 236.45927  ,
       218.90294  , 210.44525  , 197.08372  , 188.07436  , 167.67     ,
       149.60269  ,  98.58109  ,  53.62831  ,  30.867987 ,  22.470125 ,
        14.814765 ,  13.3523035,  12.265975 ,  11.324723 ,  10.879661 ,
        10.43916  ,  10.277948 ,   9.794174 ,   9.202323 ,   9.04031  ,
         8.951822 ,   8.724761 ,   8.640952 ,   8.155687 ,   8.112633 ,
         7.8740554,   7.8396683,   7.635454 ,   7.4626675,   7.3418713,
         7.146402 ,   7.120351 ,   6.8416977,   6.741557 ,   6.5816426,
         6.518615 ,   6.459937 ,   6.313223 ,   6.248057 ,   6.1986604,
         6.131592 ,   6.0007534,   5.9496264,   5.836926 ,   5.798414 ,
         5.73573  ,   5.5497375,   5.516746 ,   5.39587  ,   5.3153834,
         5.267165 ,   5.180821 ,   5.1271567,   5.0876694,   5.0275226,
         4.9591527,   4.891884 ,   4.833663 ,   4.7871876,   4.7

In [23]:
pca_embs['true_label'] = targets

In [24]:
clusters = {k : np.stack([PCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [25]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

traces = []

for (k, v) in sorted(clusters.items()):
  traces.append(go.Scatter(
                    x = v[:, 2],
                    y = v[:, 3],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))
data = traces

title = "Visualizing Clusters in Two Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [26]:
for (true, spur), idxs in valset.group_partition.items():
  for i in idxs:
    pca_embs.loc[i, 'spurious_label'] = spur
pca_embs['spurious_label'] = pca_embs['spurious_label'].astype(int)

In [27]:
valset.group_partition.keys()

dict_keys([(3, 1), (1, 1), (0, 0), (2, 0), (0, 1), (2, 4), (4, 2), (2, 1), (4, 4), (0, 3), (3, 4), (0, 4), (4, 3), (1, 3), (2, 2), (3, 3), (3, 0), (2, 3), (0, 2), (1, 4), (1, 2), (3, 2), (1, 0), (4, 1), (4, 0)])

In [28]:
true_label_clusters = {
    # True label: {group, cluster_dim}
}
for l in pca_embs['true_label'].unique():
  true_cluster = pca_embs[pca_embs['true_label'] == l]
  true_emb = true_cluster.iloc[:, :128].values
  pca = PCA(n_components=3)
  PCs = pd.DataFrame(pca.fit_transform(embeddings), columns = ['PC1_TL', 'PC2_TL', 'PC3_TL'])
  PCs = pd.concat([PCs,true_cluster[['true_label', 'spurious_label']]], axis=1, join='inner')
  true_label_clusters[l] = PCs
  # pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')


In [29]:
spurious_label_clusters = {
    # Spurious label: {group, cluster_dim}
}
for l in pca_embs['spurious_label'].unique():
  spurious_cluster = pca_embs[pca_embs['spurious_label'] == l]
  spurious_emb = spurious_cluster.iloc[:, :128].values
  pca = PCA(n_components=3)
  PCs = pd.DataFrame(pca.fit_transform(embeddings), columns = ['PC1_TL', 'PC2_TL', 'PC3_TL'])
  PCs = pd.concat([PCs,spurious_cluster[['true_label', 'spurious_label']]], axis=1, join='inner')
  spurious_label_clusters[l] = PCs
  # pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')


In [30]:
import os

save_dir = f"/content/drive/MyDrive/CS260D_final_project/plots/"
for k, v in true_label_clusters.items():
  traces = []
  for s in np.sort(v['spurious_label'].unique()):
    group = v[v['spurious_label'] == s]
    traces.append(go.Scatter(
                      x = group['PC2_TL'],
                      y = group['PC3_TL'],
                      #z = group['PC3_TL'],
                      mode = "markers",
                      name = f"Subgroup {s}",
                      text = None))
  data = traces

  title = f"Visualizing Clusters in Two Dimensions Using PCA - True Label {k}"

  layout = dict(title = title,
                xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
                yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
              )

  fig = dict(data = data, layout = layout)
  save_name = f"simCLR_{Config.spurious_correlation_strength}_LARGE_TL{k}_PCA_plot.html"
  if os.path.isdir(save_dir):
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))
  else:
    os.mkdir(save_dir)
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))

  iplot(fig)

In [31]:
import os

save_dir = f"/content/drive/MyDrive/CS260D_final_project/plots/"
for k, v in spurious_label_clusters.items():
  traces = []
  for s in np.sort(v['true_label'].unique()):
    group = v[v['true_label'] == s]
    traces.append(go.Scatter(
                      x = group['PC1_TL'],
                      y = group['PC3_TL'],
                      #z = group['PC3_TL'],
                      mode = "markers",
                      name = f"True Label {s}",
                      text = None))
  data = traces

  title = f"Visualizing Clusters in Two Dimensions Using PCA - Spurious Label {k}"

  layout = dict(title = title,
                xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
                yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
              )

  fig = dict(data = data, layout = layout)
  save_name = f"simCLR_{Config.spurious_correlation_strength}_LARGE_TL{k}_Spurious_Label_PCA_plot.html"
  if os.path.isdir(save_dir):
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))
  else:
    os.mkdir(save_dir)
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))

  iplot(fig)

## t-SNE

In [32]:
from sklearn.manifold import TSNE

perplexity = 50

#T-SNE with two dimensions
tsne_2d = TSNE(n_components=2, perplexity=perplexity)

TCs_2d = pd.DataFrame(tsne_2d.fit_transform(embeddings))
TCs_2d.columns = ["TC1_2d","TC2_2d"]
tsne_embs = pd.concat([pd.DataFrame(embeddings),TCs_2d], axis=1, join='inner')

In [33]:
tsne_embs

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,TC1_2d,TC2_2d
0,1.536616,0.195015,0.050829,0.903331,-0.740013,1.201940,0.689036,-1.504544,1.591436,-0.789175,...,0.863811,-0.113369,1.053264,-0.532731,0.292884,-0.202957,0.556975,0.182968,45.648182,-29.710785
1,0.850726,-0.127242,-0.348276,-0.189618,-0.285171,0.717334,1.141249,-2.161782,1.344776,-0.709910,...,1.338843,-0.099006,0.243697,-0.473950,0.410032,-0.174087,0.486229,-0.008911,43.524410,-18.419363
2,-1.091832,2.103783,-1.313010,0.338406,-0.345317,0.210435,-1.753662,1.225974,0.291373,-0.630485,...,-0.294803,-0.057698,-0.109906,0.486965,0.286756,-2.577257,0.339929,0.917097,-53.675339,-43.797916
3,0.009009,0.506664,-1.605244,-1.532069,-0.345650,0.059589,-0.992473,0.763059,-0.646846,-0.631010,...,0.721393,-0.886256,0.815558,-1.032609,-0.206389,-1.291324,0.345575,-0.594388,36.341503,46.563065
4,-0.132195,1.095386,-0.367323,-0.533907,1.153049,-2.317460,-1.239074,-0.129317,0.557468,-1.634831,...,-0.697408,2.694703,0.166737,0.014005,0.960820,-1.427164,0.274946,-1.336937,-27.161970,-39.779942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.554174,-0.958368,0.872555,-0.819390,-1.050047,0.989670,-0.066352,-0.279708,-0.560630,1.072883,...,-1.401734,-0.548852,0.528569,0.976648,-1.072442,1.776037,1.178261,0.251608,-21.563072,36.310867
9996,0.603031,0.545925,0.504199,0.196884,1.076206,0.683114,0.971301,0.545453,0.440392,-0.816750,...,1.067952,0.003846,-1.023273,0.100530,-0.106378,-0.272202,0.802295,-0.885886,12.966135,-8.111155
9997,0.189925,-0.433942,-2.634098,-1.119807,2.010422,-3.174433,-1.319356,0.342118,0.316112,-0.524704,...,-0.972931,1.475559,-0.589844,0.060599,2.984095,-0.714589,-0.089038,-2.152825,-26.065348,-52.628063
9998,0.029903,-0.442246,-0.533113,-1.979501,1.057163,0.641766,0.692636,1.041171,-0.046622,-0.009142,...,0.145149,0.398258,-2.020121,0.467003,0.740475,0.136426,-1.891703,0.622138,-0.739699,-1.323714


In [34]:
tsne_clusters = {k : np.stack([TCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [35]:
tsne_traces = []

for (k, v) in sorted(tsne_clusters.items()):
  tsne_traces.append(go.Scatter(
                    x = v[:, 0],
                    y = v[:, 1],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))

In [36]:
data = tsne_traces

title = "Visualizing Clusters in Two Dimensions Using t-SNE"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)